In [ ]:
!pip install spacy
!python -m spacy download es_core_news_sm

In [ ]:
!git clone https://github.com/MadisonAcosta/PyTutor.git


In [3]:
import json  # Importa la biblioteca JSON para manejar archivos en este formato
import spacy  # Importa la biblioteca spaCy para procesamiento de lenguaje natural
from collections import deque  # Importa deque para manejar una cola de memoria limitada
import random  # Importa random para seleccionar respuestas aleatorias

# Cargar intents y evaluaciones desde el archivo JSON
with open("intents.json", "r", encoding="utf-8") as file:
    intents = json.load(file)  # Carga el contenido del archivo JSON en un diccionario

# Cargar datos de celulares desde un archivo JSON
with open("celulares_limpios.json", "r", encoding="utf-8") as file:
    celulares = json.load(file)  # Carga el contenido del archivo JSON en un diccionario

# Inicializar la memoria del chatbot con un tamaño máximo de 5 mensajes
memoria_chat = deque(maxlen=5)
preferencias_usuario = {}  # Diccionario para almacenar las preferencias del usuario
inicio_recoleccion = False  # Bandera para indicar si se están recopilando preferencias

# Extraer preguntas del flujo de evaluaciones en intents.json
preguntas_flujo = intents["evaluacionesChat"][0]["responses"]
indice_pregunta = 0  # Índice para rastrear en qué pregunta del flujo está el chatbot

# Extraer patrones y respuestas de saludos del archivo intents.json
saludos_patterns = {pat.lower() for intent in intents["intents"] if intent["tag"] == "saludo" for pat in intent["patterns"]}
saludos_responses = [resp for intent in intents["intents"] if intent["tag"] == "saludo" for resp in intent["responses"]]

# Función para validar si la entrada del usuario está vacía
def validar_entrada_vacia(user_input):
    if not user_input:
        print("Chatbot: No entendí tu respuesta. Por favor, escribe algo.")
        return False
    return True

# Función para validar si el usuario quiere salir
def validar_salida(user_input):
    if user_input == "salir":
        print("Chatbot: ¡Hasta luego!")
        return True
    return False

# Función para validar si la entrada del usuario es un saludo
def validar_saldo_patterns(user_input):
    if user_input in saludos_patterns:
        print(f"Chatbot: {random.choice(saludos_responses)}")  # Responde con un saludo aleatorio
        return True
    return False

# Función para validar si el usuario ingresó un número
def validar_numero(user_input):
    try:
        return float(user_input) if "." in user_input else int(user_input)
    except ValueError:
        print("Chatbot: Por favor, ingresa un número válido.")
        return None

# Función para validar respuestas de sí o no
def validar_respuesta_si_no(user_input):
    respuestas_validas = {"sí": True, "si": True, "no": False}
    return respuestas_validas.get(user_input, None)

# Función para normalizar la entrada eliminando espacios y convirtiendo a minúsculas
def normalizar_entrada(user_input):
    return user_input.strip().lower()

# Función para validar si la marca ingresada está en la lista de celulares
def validar_marca(user_input):
    marcas_disponibles = {c["Company Name"].lower() for c in celulares}
    if user_input.lower() not in marcas_disponibles:
        print("Chatbot: No encontré esa marca. Aquí algunas opciones: ", ", ".join(marcas_disponibles))
        return None
    return user_input

# Función para detectar si el usuario está enviando spam
def detectar_spam(user_input):
    if memoria_chat.count(user_input) > 2:
        print("Chatbot: Estás repitiendo mucho el mismo mensaje. ¿Necesitas ayuda con algo específico?")
        return True
    return False

# Función para recomendar un celular basado en las preferencias del usuario
def recomendar_celular():
    celulares_filtrados = celulares  # Lista inicial de celulares
    for clave, valor in preferencias_usuario.items():
        if clave == "presupuesto":
            celulares_filtrados = [c for c in celulares_filtrados if c.get("Launched Price (USA)", float("inf")) <= valor]
        elif clave == "camara":
            celulares_filtrados = [c for c in celulares_filtrados if c.get("Front Camera (MP)", 0) >= valor]
        elif clave == "rendimiento":
            celulares_filtrados = [c for c in celulares_filtrados if c.get("RAM (GB)", 0) >= valor]
        elif clave == "bateria":
            celulares_filtrados = [c for c in celulares_filtrados if c.get("Battery Capacity (mAh)", 0) >= valor]
        elif clave == "marca":
            celulares_filtrados = [c for c in celulares_filtrados if valor.lower() in c.get("Company Name", "").lower()]
    return celulares_filtrados[0] if celulares_filtrados else None

# Función principal del chatbot
def chatbot():
    global inicio_recoleccion, preferencias_usuario, indice_pregunta
    print("Hola, ¿en qué puedo ayudarte?")
    while True:
        user_input = normalizar_entrada(input("Tú: "))  # Recibe y normaliza la entrada del usuario
        memoria_chat.append(user_input)  # Agrega la entrada a la memoria del chatbot

        if not validar_entrada_vacia(user_input) or detectar_spam(user_input):
            continue
        
        if validar_salida(user_input):
            break
        
        if validar_saldo_patterns(user_input):
            continue
        
        if not inicio_recoleccion:
            if any(word in user_input for word in ["recomendar", "celular", "jugar", "busco"]):
                print("Chatbot: Claro, dime tus preferencias.")
                inicio_recoleccion = True
                indice_pregunta = 0
                preferencias_usuario.clear()
            else:
                print("Chatbot: Lo siento, no entendí tu pregunta. ¿Puedes reformularla?")
            continue

        if indice_pregunta < len(preguntas_flujo):
            pregunta_actual = preguntas_flujo[indice_pregunta]["pregunta"]
            if "batería" in pregunta_actual.lower():
                respuesta_validada = validar_respuesta_si_no(user_input)
                if respuesta_validada:
                    preferencias_usuario["bateria"] = 4000
            elif "presupuesto" in pregunta_actual.lower():
                respuesta_validada = validar_numero(user_input)
                if respuesta_validada is not None:
                    preferencias_usuario["presupuesto"] = respuesta_validada
            indice_pregunta += 1

            if indice_pregunta < len(preguntas_flujo):
                print(f"Chatbot: {preguntas_flujo[indice_pregunta]['pregunta']}")
            else:
                celular_recomendado = recomendar_celular()
                if celular_recomendado:
                    print(f"Chatbot: Te recomiendo el {celular_recomendado['Model Name']} de {celular_recomendado['Company Name']}")
                else:
                    print("Chatbot: No encontré un celular que cumpla con todas tus preferencias.")
                inicio_recoleccion = False
                indice_pregunta = 0
                preferencias_usuario.clear()

chatbot()


Hola, ¿en qué puedo ayudarte?
Chatbot: ¡Hey! ¿Cómo estás?
Chatbot: ¡Hola! ¿En qué puedo ayudarte hoy?
Chatbot: Estás repitiendo mucho el mismo mensaje. ¿Necesitas ayuda con algo específico?
Chatbot: Estás repitiendo mucho el mismo mensaje. ¿Necesitas ayuda con algo específico?
Chatbot: Claro, dime tus preferencias.
Chatbot: ¿Cuánto es tu presupuesto máximo? (Ingresa un número)
Chatbot: ¿Qué marca prefieres? (Escribe una marca específica)
Chatbot: ¿Quieres una buena cámara? (Sí/No)
Chatbot: No entendí tu respuesta. Por favor, escribe algo.
Chatbot: ¿Necesitas un celular con buen rendimiento? (Sí/No)
Chatbot: No entendí tu respuesta. Por favor, escribe algo.
Chatbot: Te recomiendo el iphone 16 128gb de Apple
Chatbot: No entendí tu respuesta. Por favor, escribe algo.
Chatbot: No entendí tu respuesta. Por favor, escribe algo.
Chatbot: No entendí tu respuesta. Por favor, escribe algo.
Chatbot: Lo siento, no entendí tu pregunta. ¿Puedes reformularla?
Chatbot: Lo siento, no entendí tu pregunt